In [ ]:
import pandas as pd
%matplotlib inline

In [ ]:
encdata = pd.read_csv("encounters.csv.gz")

In [ ]:
encdata.drop_duplicates(subset='Id_ENC', keep='first')["AGE@ENC"].plot.hist(bins=50)

In [ ]:
encdata.drop_duplicates(subset='Id_ENC', keep='first')["RACE"].value_counts().plot.bar()

In [ ]:
encdata.drop_duplicates(subset='Id_ENC', keep='first')["GENDER"].value_counts().plot.bar()

In [ ]:
encdata.head(50)

### How many encounters do we have?

In [ ]:
len(encdata['Id_ENC'].unique())

In [ ]:
len(encdata['PATIENT'].unique())

### Select BMI Data

In [ ]:
bmi=encdata[encdata['DESCRIPTION_OBS']=='Body Mass Index']

In [ ]:
bmi.head(20)

In [ ]:
bmi = bmi.drop_duplicates(subset='Id_ENC', keep='first')
bmi.shape

## Filter based on our alert

In [ ]:
alerts = bmi[(bmi["AGE@ENC"]>45 ) | (bmi["VALUENUMERIC"]>25)]
alerts

In [ ]:
alerts.shape

## What Percentage of the Encounters Generate Our Alert?

In [ ]:
alerts.shape[0]/bmi.shape[0]

### How Many Unique Patients Did We Generate an Alert for?

In [ ]:
len(alerts["PATIENT"].unique())

## Additional data we could filter on

### What are our conditions?

In [ ]:
for d in encdata["DESCRIPTION_CON"].dropna().unique():
    print(d)

### Diabetes conditions

In [ ]:
for d in encdata["DESCRIPTION_CON"].dropna().unique():
    if 'diabetes' in d.lower():
        print(d)

### What are our observations?

In [ ]:
for o in encdata["DESCRIPTION_OBS"].dropna().unique():
    print(o)

### What are our medications?

In [ ]:
for m in encdata["DESCRIPTION_MED"].dropna().unique():
    print(m)

In [ ]:
for m in encdata["DESCRIPTION_MED"].dropna().unique():
    if 'insul' in m.lower():
        print(m)